In [ ]:
#导入相关的库
import mysql.connector
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta
from dateutil.parser import parse

In [ ]:
#数据库连接初始化
mydb = mysql.connector.connect(
    host="rm-bp178mt2593im687xjo.mysql.rds.aliyuncs.com",
    user="root",
    passwd="KLxs8888",
    database="wdgj"
)
print(mydb)

In [ ]:
# #数据库连接初始化
# mydb = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     passwd="qaz!@wsx",
#     database="work"
# )
# print(mydb)

In [ ]:
#定义SQL语句
profitCheck='SELECT * FROM profit_check_sku_shop'
returnDetail ='SELECT * FROM return_sku_shop'

In [ ]:
#打开游标，获取数据
mycursor=mydb.cursor()
mycursor.execute(profitCheck)
profitCheck1901=mycursor.fetchall()
mycursor.close()

In [ ]:
#打开游标，获取数据
mycursor=mydb.cursor()
mycursor.execute(returnDetail)
returnSku1901=mycursor.fetchall()
mycursor.close()

In [ ]:
#数据格式转化成为pandas
profitCheck1901=pd.DataFrame(profitCheck1901)
returnSku1901=pd.DataFrame(returnSku1901)

In [ ]:
profitCheck1901.columns=['Date','GoodsNO','BarCode','GoodsName','GoodsCount','Income','Cost','FavourableTotal'\
                        ,'Postage','PostageTotal','OperationCost','shopname']
returnSku1901.columns=['Date','GoodsNO','GoodsName','ReturnTotal','GoodsCount','returnCost','shopname']

In [ ]:
returnSku1901.head()

In [ ]:
returnSku1901["GoodsCount"]=returnSku1901["GoodsCount"].astype(float)
result2=returnSku1901.groupby(['Date'],as_index=False)
result2=result2.aggregate(np.sum)#出库总数
result2.GoodsCount.sum()

In [ ]:
profitCheck1901.head()

In [ ]:
returnSku1901["GoodsCount"]=returnSku1901["GoodsCount"].astype(float)

In [ ]:
#合并出库明细和退货明细
data_merge = pd.merge(profitCheck1901,returnSku1901,on=['Date','GoodsNO','shopname'],how='left')

In [ ]:
returnSku1901["GoodsCount"]=returnSku1901["GoodsCount"].astype(float)
data_merge1=data_merge.groupby(['Date'],as_index=False)
data_merge1=data_merge1.aggregate(np.sum)#出库总数
data_merge1.GoodsCount_y.sum()

In [ ]:
# data_merge.to_excel("data_merge_right.xlsx",header=True)

In [ ]:
data_merge.head()

In [ ]:
#data_merge.to_excel("data_merge.xlsx",header=True,index=False)

In [ ]:
data_merge.head()

In [ ]:
del data_merge['GoodsName_y']

In [ ]:
#获取商品小组对应情况
GoodsInfo =pd.read_csv('C:\\Users\\Administrator\\PycharmProjects\\test\\goodsinfo0402.csv',header = 0,engine='python')
GoodsInfo.columns=['GoodsNO','GoodsName','BarCode','Company','Dept','TmallTax','GroupName','Holder','GradeTwo','GradeThird','GradeForth','SPU']

In [ ]:
#删除重复以及不必要列
GoodsInfoShort=GoodsInfo[['Company','Dept','GroupName','TmallTax','Holder','GradeTwo','GradeThird','GradeForth','SPU','BarCode']]
# GoodsInfoShort=GoodsInfo[['Company','Dept','GroupName','TmallTax','Holder','GradeTwo','GradeThird','GradeForth','SPU','GoodsNO']]

In [ ]:
GoodsInfoShort.shape

In [ ]:
#以SKU编码为唯一辨识，去重
# GoodsInfoShort=GoodsInfoShort.drop_duplicates(["GoodsNO"])
#以BarCode为唯一辨识，去重
GoodsInfoShort=GoodsInfoShort.drop_duplicates(["BarCode"])

In [ ]:
GoodsInfoShort.shape

In [ ]:
GoodsInfoShort.head()

In [ ]:
#匹配销售数据和对应小组信息
data_merge_dept=pd.merge(data_merge,GoodsInfoShort,on=['BarCode'],how='left')

In [ ]:
# data_merge_dept = pd.read_csv('C:\\Users\\Administrator\\Desktop\\sell_return_dept.csv')

In [ ]:
data_merge_dept.head()

In [ ]:
data_merge_dept.columns

In [ ]:
# kub=pd.read_csv('C:\\Users\\Administrator\\Desktop\\商品SKU事业部\\0124\\kub0125.csv',engine='python')
# kub=kub[['GoodsNO','Dept','GroupName']]
# kub=kub.drop_duplicates(["GoodsNO"])
# kub.columns=['SKU','Dept','GroupName']
# data_merge_dept_gp=pd.merge(data_merge_dept,kub,on=['SKU'],how='left')
# data_merge_dept_gp.to_excel('data_group_new0125.xlsx',header=True,index=False)

In [ ]:
#修改合并后的列名
data_merge_dept.columns=['Date', 'SKU', 'BarCode', 'Name', 'SellCount', 'SellIncome',
       'SellCost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost','ShopName',
       'ReturnTotal', 'ReturnCount', 'ReturnCost', 'Company', 'Dept',
       'GroupName','TmallTax','Holder','GradeTwo','GradeThird','GradeForth','SPU']

In [ ]:
data_merge_dept.head()

In [ ]:
shopname={'贝豪婴童专营店':'天猫商城',
'欣懿宝贝-米歌':'天猫商城',
'可优比旗舰店':'天猫商城',
'蒂爱旗舰店':'天猫商城',
'贝豪拼多多':'拼多多',
'麦可适旗舰店':'天猫商城',
'可优比（杭州来福士）':'线下',
'唯品会（蒂爱）':'唯品会',
'美易仓库':'美易仓库',
'京东可优比母婴旗舰店':'京东商城',
'可优比海外自营店':'海外店',
'可优比天猫超市':'天猫超市',
'唯品会（可优比）':'唯品会',
'唯品会MP（蒂爱）':'唯品会',
'优趣玩具专营店':'天猫商城',
'可优比母婴生活馆':'有赞',
'贝豪贝店商城':'贝贝网',
'可优比团购店铺':'团购',
'蒂爱团购店铺':'团购',
'京东蒂爱母婴旗舰店':'京东商城'}

In [ ]:
data_merge_dept['Source']=data_merge_dept['ShopName'].map(shopname)

In [ ]:
#data_merge_dept['Postage']=0

In [ ]:
data_merge_dept['GroupName']=data_merge_dept['GroupName'].fillna('#N/A')

In [ ]:
data_merge_dept=data_merge_dept.drop_duplicates(["SKU","ShopName"])

In [ ]:
data_merge_dept[['SellCount', 'SellIncome',
       'SellCost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost',
       'ReturnTotal', 'ReturnCount', 'ReturnCost','TmallTax']] = data_merge_dept[['SellCount', 'SellIncome',
       'SellCost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost',
       'ReturnTotal', 'ReturnCount', 'ReturnCost','TmallTax']].astype(float)

In [ ]:
data_merge_dept.to_csv("C:\\Users\\Administrator\\Desktop\\data_merge_dept_all.csv",header=True,index=False)
data_merge_dept.to_excel("C:\\Users\\Administrator\\Desktop\\data_merge_dept_all.xlsx",header=True,index=False)

In [ ]:
data_merge_dept=data_merge_dept.dropna(axis=0,subset=['Source','Dept'])
data_merge_dept=data_merge_dept[(data_merge_dept['Dept']!='KUB包材')&(data_merge_dept['Dept']!='KUB赠品')\
                          &(data_merge_dept['Dept']!='蒂爱共用')&(data_merge_dept['Source']!='NaN')]

In [ ]:
data_merge_dept[data_merge_dept['SKU']=='109804']['Source']

In [ ]:
data_merge_dept.to_csv("C:\\Users\\Administrator\\Desktop\\data_merge_dept.csv",header=True,index=False)
data_merge_dept.to_excel("C:\\Users\\Administrator\\Desktop\\data_merge_dept.xlsx",header=True,index=False)

In [ ]:
department_gross_profit = pd.DataFrame()
department_gross_profit = pd.concat([department_gross_profit,data_merge_dept])

In [ ]:
#查看缺失值情况
department_gross_profit=department_gross_profit.fillna(np.nan)
department_gross_profit.isnull().sum()

In [ ]:
department_gross_profit.head()

In [ ]:
department_gross_profit[['SellCount', 'SellIncome',
       'SellCost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost',
       'ReturnTotal', 'ReturnCount', 'ReturnCost','TmallTax']] = department_gross_profit[['SellCount', 'SellIncome',
       'SellCost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost',
       'ReturnTotal', 'ReturnCount', 'ReturnCost','TmallTax']].astype(float)

In [ ]:
department_gross_profit.head()

In [ ]:
#填充缺失值为0
department_gross_profit[['ReturnTotal', 'ReturnCount', 'ReturnCost','TmallTax']]=department_gross_profit\
[['ReturnTotal', 'ReturnCount', 'ReturnCost','TmallTax']].fillna(0)

#填充缺失小组为#N/A
department_gross_profit['GroupName']=department_gross_profit\
['GroupName'].fillna('#N/A')

In [ ]:
department_gross_profit['CheckIncome'] = department_gross_profit['SellIncome']-department_gross_profit['FavourableTotal']\
                                        +department_gross_profit['PostageTotal']-department_gross_profit['ReturnTotal']
department_gross_profit['CheckCost'] = department_gross_profit['SellCost']-department_gross_profit['ReturnCost']
department_gross_profit['TmallCharge'] = (department_gross_profit['SellIncome']-department_gross_profit['FavourableTotal']-\
                                        department_gross_profit['ReturnTotal'])*department_gross_profit['TmallTax']

In [ ]:
# #按公司，部门，日期进行归类
department_gross_profit_gb=department_gross_profit.groupby(['Company','Dept','Date','Source','ShopName'],as_index=False)

In [ ]:
#按公司，部门，日期进行归类
# department_gross_profit_gb=department_gross_profit.groupby(['Company','Dept','Date'],as_index=False)

In [ ]:
#归类汇总
department_gross_profit_gb = department_gross_profit_gb.aggregate(np.sum)
del department_gross_profit_gb['TmallTax']

In [ ]:
#计算营业税，毛利润，毛利率
department_gross_profit_gb['Tax']=department_gross_profit_gb['CheckIncome']*0.06
department_gross_profit_gb['GrossProfit']=department_gross_profit_gb['CheckIncome']-department_gross_profit_gb['CheckCost']-\
                                            department_gross_profit_gb['CheckIncome']*0.06-department_gross_profit_gb['TmallCharge']-\
                                            department_gross_profit_gb['Postage']-department_gross_profit_gb['OperationCost']
department_gross_profit_gb['GrossProfitRate']=department_gross_profit_gb['GrossProfit']/department_gross_profit_gb['SellIncome']

In [ ]:
department_gross_profit_gb=department_gross_profit_gb[(department_gross_profit_gb['Dept']!='KUB包材')&(department_gross_profit_gb['Dept']!='KUB赠品')\
                          &(department_gross_profit_gb['Dept']!='蒂爱共用')]

In [ ]:
department_gross_profit_gb.head()

In [ ]:
department_gross_profit_gb.columns

In [ ]:
department_gross_profit_gb.to_csv("C:\\Users\\Administrator\\Desktop\\department_gross_profit_gb.csv",header=True,index=False)

In [ ]:
department_gross_profit_gb.columns=['公司','事业部','日期','渠道','店铺','销售数量','销售收入','销售货品成本','优惠金额','快递费','应收邮资','操作费','退货金额','退货数量','退货货品成本','核算收入','核算成本','天猫扣点','营业税','毛利润','毛利率']
department_gross_profit_gb.to_excel("C:\\Users\\Administrator\\Desktop\\department_gross_profit_gb.xlsx",header=True,index=False)

In [ ]:
# department_gross_profit_gb.columns=['公司','事业部','日期','销售数量','销售收入','销售货品成本','优惠金额','快递费','应收邮资','操作费','退货金额','退货数量','退货货品成本','核算收入','核算成本','天猫扣点','营业税','毛利润','毛利率']
# department_gross_profit_gb.to_excel("C:\\Users\\Administrator\\Desktop\\department_gross_profit_gb.xlsx",header=True,index=False)

In [ ]:
# data_merge1=department_gross_profit_gb.groupby(['Date'],as_index=False)
# data_merge1=data_merge1.aggregate(np.sum)#出库总数
# data_merge1.ReturnCount.sum()

In [ ]:
#按公司、部门、小组已经日期进行归类
department_gross_profit_gb_group=department_gross_profit.groupby(['Company','Dept','GroupName','Date','Source','ShopName'],as_index=False)
# department_gross_profit_gb_group=department_gross_profit.groupby(['Company','Dept','GroupName','Date'],as_index=False)

In [ ]:
department_gross_profit_gb_group = department_gross_profit_gb_group.aggregate(np.sum)
del department_gross_profit_gb_group['TmallTax']

In [ ]:
department_gross_profit_gb_group.head()

In [ ]:
department_gross_profit_gb_group['Tax']=department_gross_profit_gb_group['CheckIncome']*0.06
department_gross_profit_gb_group['GrossProfit']=department_gross_profit_gb_group['CheckIncome']-department_gross_profit_gb_group['CheckCost']-\
                                            department_gross_profit_gb_group['CheckIncome']*0.06-department_gross_profit_gb_group['TmallCharge']-\
                                            department_gross_profit_gb_group['Postage']-department_gross_profit_gb_group['OperationCost']
department_gross_profit_gb_group['GrossProfitRate']=department_gross_profit_gb_group['GrossProfit']/department_gross_profit_gb_group['SellIncome']

In [ ]:
department_gross_profit_gb_group=department_gross_profit_gb_group[(department_gross_profit_gb_group['Dept']!='KUB包材')&(department_gross_profit_gb_group['Dept']!='KUB赠品')\
                          &(department_gross_profit_gb_group['Dept']!='蒂爱共用')]

In [ ]:
department_gross_profit_gb_group.head()

In [ ]:
department_gross_profit_gb_group.to_csv("C:\\Users\\Administrator\\Desktop\\department_gross_profit_gb_group.csv",header=True,index=False)
department_gross_profit_gb_group.to_excel("C:\\Users\\Administrator\\Desktop\\department_gross_profit_gb_group.xlsx",header=True,index=False)

In [ ]:
data_merge_dept_edit=pd.DataFrame()
data_merge_dept_edit=pd.concat([data_merge_dept_edit,data_merge_dept])

In [ ]:
# data_merge_dept_edit.columns=['日期','SKU编码','条码','品名','销售数量','销售收入','货品成本','优惠金额','快递费',\
#                         '应收邮资','操作费','店铺名称','退货金额','退货数量','退货货品成本','公司名称','事业部','小组','天猫扣点','负责人','二级类目','三级类目','四级类目','SPU','渠道']

In [ ]:
data_merge_dept_edit.columns

In [ ]:
#填充缺失值为0
data_merge_dept_edit=data_merge_dept_edit.fillna(np.nan)
data_merge_dept_edit.isnull().sum()

In [ ]:
# data_merge_dept_edit[['操作费','退货金额','退货数量','退货货品成本']]=data_merge_dept_edit\
# [['操作费','退货金额','退货数量','退货货品成本']].fillna(0)

In [ ]:
# data_merge_dept_edit.to_excel("20190131.xlsx",header=True,index=False)
data_merge_dept_edit.to_excel("C:\\Users\\Administrator\\Desktop\\data_merge_dept.xlsx",header=True,index=False)

In [ ]:
# data_merge_dept.columns=['Date', 'SKU', 'BarCode', 'Name', 'SellCount', 'SellIncome',
#        'SellCost', 'FavourableTotal', 'Postage', 'PostageTotal', 'OperationCost',
#        'ReturnTotal', 'ReturnCount', 'ReturnCost', 'Company', 'Dept',
#        'Group']

In [ ]:
# def merge(data_left,data_right,key):
#     left=pd.read_csv(data_left,header = 0,dtype = object,engine='python',encoding='utf-8')
#     right=pd.read_csv(data_right,header = 0,dtype = object,engine='python')
#     right=right.drop_duplicates(key)
#     data_merge=pd.merge(left,right,on=key,how='left')
#     return data_merge

In [ ]:
# del data['BarCode_x']
# data['BarCode']=data['BarCode_y']
# data=data.drop('BarCode_y',axis=1)

In [ ]:
data_merge_dept.head()

In [ ]:
returnSku1901.head()
returnSku1901.columns=['Date', 'SKU', 'Name', 'ReturnTotal', 'GoodsCount','returnCost', 'ShopName']
returnSku1901.head()

In [ ]:
return_diff=pd.merge(returnSku1901,data_merge_dept,on=['Date','Name','ShopName','SKU'],how='left')

In [ ]:
return_diff.columns
return_diff=return_diff[['Date', 'SKU', 'Name', 'ReturnTotal_x', 'GoodsCount',
       'returnCost', 'ShopName','ReturnTotal_y', 'ReturnCount', 'ReturnCost']]

In [ ]:
return_diff.to_excel("C:\\Users\\Administrator\\Desktop\\return_diff.xlsx",header=True,index=False)

In [ ]:
returnSku1901["GoodsCount"]=returnSku1901["GoodsCount"].astype(float)
result2=returnSku1901.groupby(['Date'],as_index=False)
result2=result2.aggregate(np.sum)#出库总数
result2.GoodsCount.sum()